In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('HWApp').getOrCreate()

In [15]:
fw=spark.read.csv('d:\\wh.csv',header=True,inferSchema=True)
fw.show()

+------+-----------+-----------+
|Gender|     Height|     Weight|
+------+-----------+-----------+
|  Male|73.84701702|241.8935632|
|  Male|68.78190405|162.3104725|
|  Male|74.11010539|212.7408556|
|  Male| 71.7309784|220.0424703|
|  Male|69.88179586|206.3498006|
|  Male|67.25301569|152.2121558|
|  Male|68.78508125|183.9278886|
|  Male|68.34851551|167.9711105|
|  Male|67.01894966|175.9294404|
|  Male|63.45649398|156.3996764|
|  Male|71.19538228|186.6049256|
|  Male|71.64080512|213.7411695|
|  Male|64.76632913|167.1274611|
|  Male| 69.2830701|189.4461814|
|  Male|69.24373223| 186.434168|
|  Male| 67.6456197|172.1869301|
|  Male|72.41831663|196.0285063|
|  Male|63.97432572|172.8834702|
|  Male| 69.6400599|185.9839576|
|  Male|67.93600485| 182.426648|
+------+-----------+-----------+
only showing top 20 rows



In [5]:
fw.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)



In [16]:
#independent feature gender col is non numeric so we will import stringindexer
from pyspark.ml.feature import StringIndexer

In [17]:
g=StringIndexer(inputCol='Gender',outputCol='Gen_Col')

In [18]:
v=g.fit(fw).transform(fw)

In [19]:
v.show()

+------+-----------+-----------+-------+
|Gender|     Height|     Weight|Gen_Col|
+------+-----------+-----------+-------+
|  Male|73.84701702|241.8935632|    1.0|
|  Male|68.78190405|162.3104725|    1.0|
|  Male|74.11010539|212.7408556|    1.0|
|  Male| 71.7309784|220.0424703|    1.0|
|  Male|69.88179586|206.3498006|    1.0|
|  Male|67.25301569|152.2121558|    1.0|
|  Male|68.78508125|183.9278886|    1.0|
|  Male|68.34851551|167.9711105|    1.0|
|  Male|67.01894966|175.9294404|    1.0|
|  Male|63.45649398|156.3996764|    1.0|
|  Male|71.19538228|186.6049256|    1.0|
|  Male|71.64080512|213.7411695|    1.0|
|  Male|64.76632913|167.1274611|    1.0|
|  Male| 69.2830701|189.4461814|    1.0|
|  Male|69.24373223| 186.434168|    1.0|
|  Male| 67.6456197|172.1869301|    1.0|
|  Male|72.41831663|196.0285063|    1.0|
|  Male|63.97432572|172.8834702|    1.0|
|  Male| 69.6400599|185.9839576|    1.0|
|  Male|67.93600485| 182.426648|    1.0|
+------+-----------+-----------+-------+
only showing top

In [20]:
fw.na.drop(how='any')

DataFrame[Gender: string, Height: double, Weight: double]

In [11]:
fw.count()

10000

In [21]:
from pyspark.ml.feature import VectorAssembler

In [22]:
va=VectorAssembler(inputCols=['Height','Gen_Col'],outputCol='inputfeat')
p=va.transform(v)

In [23]:
p.show(5,False)

+------+-----------+-----------+-------+-----------------+
|Gender|Height     |Weight     |Gen_Col|inputfeat        |
+------+-----------+-----------+-------+-----------------+
|Male  |73.84701702|241.8935632|1.0    |[73.84701702,1.0]|
|Male  |68.78190405|162.3104725|1.0    |[68.78190405,1.0]|
|Male  |74.11010539|212.7408556|1.0    |[74.11010539,1.0]|
|Male  |71.7309784 |220.0424703|1.0    |[71.7309784,1.0] |
|Male  |69.88179586|206.3498006|1.0    |[69.88179586,1.0]|
+------+-----------+-----------+-------+-----------------+
only showing top 5 rows



In [24]:
finaldata=p.select('inputfeat','weight')

In [25]:
finaldata.printSchema()

root
 |-- inputfeat: vector (nullable = true)
 |-- weight: double (nullable = true)



In [26]:
finaldata.show(5,False)

+-----------------+-----------+
|inputfeat        |weight     |
+-----------------+-----------+
|[73.84701702,1.0]|241.8935632|
|[68.78190405,1.0]|162.3104725|
|[74.11010539,1.0]|212.7408556|
|[71.7309784,1.0] |220.0424703|
|[69.88179586,1.0]|206.3498006|
+-----------------+-----------+
only showing top 5 rows



In [27]:
train,test=finaldata.randomSplit([.70,.30])

In [28]:
from pyspark.ml.regression import LinearRegression

In [29]:
lr=LinearRegression(featuresCol='inputfeat',labelCol='weight')
lrt=lr.fit(train)

In [30]:
res=lrt.evaluate(test)
res.predictions.show(5,False)

D:\New folder\Python\envs\envr1\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----------------+-----------+-----------------+
|inputfeat        |weight     |prediction       |
+-----------------+-----------+-----------------+
|[55.6518916,0.0] |85.62177644|87.84012769535548|
|[55.97919788,0.0]|85.41753362|89.79418998984562|
|[56.06663635,0.0]|89.57120474|90.31620935750004|
|[56.10536959,0.0]|87.29886913|90.54745198390924|
|[56.1089021,0.0] |80.53125938|90.5685415413596 |
+-----------------+-----------+-----------------+
only showing top 5 rows



In [33]:
print('r2',res.r2)
print('square of error',res.meanAbsoluteError)
print('root mean square error',res.rootMeanSquaredError)

r2 0.9068524568876848
square of error 8.019663060859644
root mean square error 10.038568924000826
